# The Battle of Neighborhoods

## Introduction
#### Shake Shack, a burger brand established in New York in 2004 is now very popular. Not only famous in its domestic market, It also has overseas branches in Asia, Middle East, and Britain. However, there is no branches in their nearest neighbor, Canada. This project aimed at comparing New York and Toronto, and look for a suitable area for Shake Shack to open up a new branch.

## Data

#### In this project, we first explore Shake Shack in New York by getting Shake Shack's nearby Venues. Then, we explore nearby Venues in each Borough of Toronto. Finally, we'll try to explore similarities between Shake Shack and each Borough in Toronto.

1. Get New York location data by geopy

In [1]:
import pandas as pd

In [2]:
!pip install geopy
from geopy.geocoders import Nominatim

address = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)



40.7127281 -74.0060152


2. Search Shake Shack in New York City

In [3]:
import requests # library to handle requests

CLIENT_ID = '4T3J520CTWULGVMCH2QF4VPKJ2BASI3JPL1SUYECDNBJ5HF0' # your Foursquare ID
CLIENT_SECRET = 'W3N5PZQXWNYTH13HBNRQ32PSBPZHGQSI1QB1S5TC2TPS03RU' # your Foursquare Secret
ACCESS_TOKEN = 'VXJXCGW41ZUOOKB5RFBEKEIWEOHNV2ZGXXP1T4JHOEDSLMAO' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 10000 # define radius

search_query = 'Shake Shack'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format( 
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude,
    ACCESS_TOKEN, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [4]:
from pandas.io.json import json_normalize
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

shake_shack = dataframe_filtered[dataframe_filtered['name'] == 'Shake Shack']
shake_shack = shake_shack[shake_shack['city']=='New York']
shake_shack

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Shake Shack,Burger Joint,200 Broadway,US,New York,United States,at Fulton St,339,"[200 Broadway (at Fulton St), New York, NY 10038]","[{'label': 'display', 'lat': 40.71070286896417...",40.710703,-74.009024,NaN,10038,NY,5787b68e498efcabbebba4f8
1,Shake Shack,Burger Joint,215 Murray St,US,New York,United States,btwn West St & North End Ave,791,"[215 Murray St (btwn West St & North End Ave),...","[{'label': 'display', 'lat': 40.71532572032048...",40.715326,-74.014750,NaN,10282,NY,4dda977ad22d38ef4332c419
3,Shake Shack,Burger Joint,"E 23rd St and Madison Avenue, Madison Square Park",US,New York,United States,Madison Ave & E 23rd St,3535,"[E 23rd St and Madison Avenue, Madison Square ...","[{'label': 'display', 'lat': 40.74148371088094...",40.741484,-73.988218,Flatiron,10010,NY,40e74880f964a520150a1fe3
6,Shake Shack,Burger Joint,691 8th Ave,US,New York,United States,at W 44th St,5276,"[691 8th Ave (at W 44th St), New York, NY 10036]","[{'label': 'display', 'lat': 40.75833417228198...",40.758334,-73.988988,Theater District,10036,NY,4c2bccc7d1a10f475a88f864
7,Shake Shack,Burger Joint,1700 Broadway,US,New York,United States,West 53rd Street,5991,"[1700 Broadway (West 53rd Street), New York, N...","[{'label': 'display', 'lat': 40.763495, 'lng':...",40.763495,-73.982419,NaN,10019,NY,5d948990f6b75b0007ab3ed2
8,Shake Shack,Burger Joint,152 E 86th St,US,New York,United States,btwn Lexington & 3rd Ave,8563,"[152 E 86th St (btwn Lexington & 3rd Ave), New...","[{'label': 'display', 'lat': 40.77909852079875...",40.779099,-73.954686,NaN,10028,NY,4c1918c7fe5a76b07b0f0315
9,Shake Shack,Burger Joint,366 Columbus Ave,US,New York,United States,at W 77th St,7964,"[366 Columbus Ave (at W 77th St), New York, NY...","[{'label': 'display', 'lat': 40.78067635406833...",40.780676,-73.976443,NaN,10024,NY,49b0c608f964a520cd521fe3
10,Shake Shack,Burger Joint,20 3rd Ave,US,New York,United States,at E 9th St,2364,"[20 3rd Ave (at E 9th St), New York, NY 10003]","[{'label': 'display', 'lat': 40.72999845323382...",40.729998,-73.989696,NaN,10003,NY,59d36de20fe7a024363de0b8
11,Shake Shack,Burger Joint,"49 Grand Central Terminal, Lower Level Dining ...",US,New York,United States,NaN,5069,"[49 Grand Central Terminal, Lower Level Dining...","[{'label': 'display', 'lat': 40.752651, 'lng':...",40.752651,-73.977108,NaN,10017,NY,51ca35d4498e950b67e87df8
12,Shake Shack,Burger Joint,"1 Penn Plaza, Lower Concourse",US,New York,United States,NaN,4433,"[1 Penn Plaza, Lower Concourse, New York, NY 1...","[{'label': 'display', 'lat': 40.75095441720344...",40.750954,-73.991275,NaN,10119,NY,5811375238fa65669ad8deba


3. Visualize Shake Shack in New York

In [5]:
import folium

venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(shake_shack.lat, shake_shack.lng, shake_shack.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

4. Get Shake Shack Nearby Venues

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

shake_shack_venues = getNearbyVenues(names=shake_shack['name'],
                                 latitudes=shake_shack['lat'],
                                 longitudes=shake_shack['lng']
                                )

Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack


In [7]:
shake_shack_venues.head()

,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Shake Shack,40.710703,-74.009024,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,Shake Shack,40.710703,-74.009024,Pisillo Italian Panini,40.710530,-74.007526,Sandwich Place
2,Shake Shack,40.710703,-74.009024,"The Beekman, A Thompson Hotel",40.711173,-74.006702,Hotel
3,Shake Shack,40.710703,-74.009024,For Five Coffee Roasters,40.709554,-74.010576,Coffee Shop
4,Shake Shack,40.710703,-74.009024,Joe’s Pizza,40.710178,-74.007769,Pizza Place


5. Get Toronto Borough Location

In [8]:
from bs4 import BeautifulSoup
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df=df[['PostalCode', 'Borough', 'Neighborhood']]

locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
toronto_data = df.join(locgeo_df, on='PostalCode')

toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


6. Visualize Toronto

In [9]:
toronto_map = folium.Map(location=[toronto_data['Latitude'].mean(), toronto_data['Longitude'].mean()],
                     zoom_start=12, tiles='Stamen Terrain')

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=1,
        parse_html=False).add_to(toronto_map)
    
toronto_map

7. Get Toronto Borough Nearby Venues

In [10]:
toronto_venues = getNearbyVenues(names=toronto_data['Borough'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                )

North York
North York
Downtown Toronto
North York
Queen's Park
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York/East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
W

In [11]:
toronto_venues.head()

,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


8. Group together and one hot encoding

In [12]:
grouped = toronto_venues.append(shake_shack_venues)

onehot_encoded = pd.get_dummies(grouped[['Venue Category']], prefix="", prefix_sep="")

onehot_encoded['Name'] = grouped['Name'] 

fixed_columns = [onehot_encoded.columns[-1]] + list(onehot_encoded.columns[:-1])
onehot_encoded = onehot_encoded[fixed_columns]

grouped = onehot_encoded.groupby('Name').mean().reset_index()

grouped

,Name,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.009009,...,0.000000,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.000000,0.009009
1,Downtown Toronto,0.000000,0.00092,0.00000,0.000920,0.00092,0.00184,0.00276,0.00092,0.011960,...,0.011040,0.001840,0.000000,0.003680,0.000000,0.007360,0.000920,0.000920,0.000000,0.004600
2,Downtown Toronto Stn A,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309
3,East Toronto,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.029126,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
4,East Toronto Business,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
5,East York,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.014085,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.014085
6,East York/East Toronto,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Etobicoke,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.015625,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000
8,Etobicoke Northwest,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Mississauga,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


9. KMeans Clustering

In [13]:
from sklearn.cluster import KMeans

kclusters = 14

grouped_clustering = grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([13,  6,  6,  9,  5, 10,  0,  8,  2,  4,  1,  7, 11,  1, 12,  3])

10. Add labels back

In [14]:
grouped.insert(0, 'Cluster Labels', kmeans.labels_)

grouped

,Cluster Labels,Name,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,13,Central Toronto,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.000000,0.009009
1,6,Downtown Toronto,0.000000,0.00092,0.00000,0.000920,0.00092,0.00184,0.00276,0.00092,...,0.011040,0.001840,0.000000,0.003680,0.000000,0.007360,0.000920,0.000920,0.000000,0.004600
2,6,Downtown Toronto Stn A,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309
3,9,East Toronto,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
4,5,East Toronto Business,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
5,10,East York,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.014085,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.014085
6,0,East York/East Toronto,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,8,Etobicoke,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000
8,2,Etobicoke Northwest,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,4,Mississauga,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


11. Merge venues

In [15]:
venues=toronto_venues.append(shake_shack_venues)
merged = venues.join(grouped.set_index('Name'), on='Name')
merged = merged[['Name','Cluster Labels','Latitude','Longitude']]
merged = merged.groupby('Name').mean().reset_index()
merged

,Name,Cluster Labels,Latitude,Longitude
0,Central Toronto,13,43.698631,-79.397703
1,Downtown Toronto,6,43.652234,-79.381877
2,Downtown Toronto Stn A,6,43.646435,-79.374846
3,East Toronto,9,43.670279,-79.338488
4,East Toronto Business,5,43.662744,-79.321558
5,East York,10,43.706382,-79.347744
6,East York/East Toronto,0,43.685347,-79.338106
7,Etobicoke,8,43.652527,-79.539176
8,Etobicoke Northwest,2,43.706748,-79.594054
9,Mississauga,4,43.636966,-79.615819


12. Visualize Maps

In [16]:
# create map
map_clusters = folium.Map(location=[merged['Latitude'].mean(), merged['Longitude'].mean()],
                     zoom_start=6, tiles='Stamen Terrain')

import numpy as np
from matplotlib import cm
from matplotlib import colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Name'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters